In [1]:
import os
import sys
import time

import ipywidgets as widgets

from openvino.inference_engine import IECore
from tqdm import tqdm_notebook as tqdm

In [17]:
inf_count = 0

def callback(status, pydata):
    global inf_count
    inf_count += 1

def benchmark(model, device='CPU', niter=1000, nireq=4):

    global inf_count
    base, ext = os.path.splitext(model)
    ie = IECore()

    if device == 'CPU':
        config = { 'CPU_BIND_THREAD'         :'YES',
                   'CPU_THREADS_NUM'         :'8', 
                   'CPU_THROUGHPUT_STREAMS'  :'4',
                   'EXCLUSIVE_ASYNC_REQUESTS':'NO' }
    else:
        config = {}

    net = ie.read_network(base+'.xml', base+'.bin')
    exenet = ie.load_network(net, device, config=config, num_requests=nireq)
    """
    met_keys = exenet.get_metric('SUPPORTED_METRICS')
    cfg_keys = exenet.get_metric('SUPPORTED_CONFIG_KEYS')
    for key in cfg_keys:
        print('{:25} : {}'.format(key, exenet.get_config(key)) )
    """

    async_api = True

    _, model = os.path.split(base)

    if async_api:
        # Set callback function
        for req in exenet.requests:
            req.set_completion_callback(callback)

    pbar = tqdm(total=niter)
    pbar.set_description('{} ({})'.format(model, device))

    inf_count = 0
    start = time.perf_counter()
    if async_api:
        # Do inference
        for i in range(niter):
            req=-1
            while req==-1:
                req = exenet.get_idle_request_id()
            exenet.requests[req].async_infer()
            pbar.update(1)
        # Wait for completion of all infer requests
        while inf_count < niter:
            pass
    else:
        for i in range(niter):
            exenet.infer()
            pbar.update(1)
    end = time.perf_counter()

    duration_sec = (end-start)
    print('Total {:5.2f}sec, {:5.2f}ms/inf, {:5.2f}fps'.format(duration_sec, (duration_sec*1e3)/niter, niter/(duration_sec)) )

def pressed(btn):
    global device_sel, model_sel, start_btn, numiter
    selected_model = model_sel.value
    model = 'public/{}/FP16/{}.xml'.format(selected_model, selected_model)
    niter = numiter.value
    device = device_sel.value
    benchmark(model, niter=niter, device=device)


model_sel = widgets.Select(
    options = [ 'googlenet-v1', 'resnet-50' ],
    value = 'googlenet-v1',
    description = 'Model',
    disabled=False)

device_sel = widgets.Select(
    options = [ 'CPU', 'GPU', 'MYRIAD', 'HETERO:FPGA,CPU' ],
    value = 'CPU',
    description = 'Device',
    disabled=False)

start_btn = widgets.Button(
    description = 'Run Benchmark',
    disabled=False,
    button_style='',
    tooltip='Press to start benchmarking',
    icon='')

numiter = widgets.BoundedIntText(
    value=1000,
    min=0,
    max=100000,
    step=1,
    description='Iteration:',
    disabled=False
)

display(device_sel)
display(model_sel)
display(numiter)
display(start_btn)
start_btn.on_click(pressed)

Select(description='Device', options=('CPU', 'GPU', 'MYRIAD', 'HETERO:FPGA,CPU'), value='CPU')

Select(description='DL model', options=('googlenet-v1', 'resnet-50'), value='googlenet-v1')

BoundedIntText(value=1000, description='Iteration:', max=100000)

Button(description='Run Benchmark', style=ButtonStyle(), tooltip='Press to start benchmarking')

Total 12.22sec, 12.22ms/inf, 81.82fps


Total  3.03sec, 30.29ms/inf, 33.01fps
